# ASTR596, FDS: Homework set 3 - Moving from MLE and Frequentism to evaluating the Posterior distribution and Bayesian Statistics
Alex Gagliano

Due: 02/19/2020


## Problem 1

Last week you showed that Bayesian and Frequentist approaches are often equivalent for simple problems (after all you implicitly had a prior in the form of a grid). 

But it is also true that they can diverge greatly. In practice, this divergence makes itself most clear in two different situations:

1. The handling of "nuisance parameters"
2. The subtle (and often overlooked) difference between frequentist confidence intervals and Bayesian credible intervals

Here, we focus on the first point: the difference between frequentist and Bayesian treatment of nuisance parameters.


## What is a Nuisance Parameter?

***A nuisance parameter is any quantity whose value is not relevant to the goal of an analysis, but is nevertheless required to determine some quantity of interest***.

For example, last week, in problem 3, we estimated both the mean $C$ and shape $\sigma_\text{source}$ for some spherical cow galaxy. Now the shape might be important, but in homework 1, you only needed the brightness of the sources where you plotted up $r$ vs $g-i$ for galaxies (you needed the shape to separate stars from galaxies in the form of MEAN_OBJECT_TYPE, but that's it).

As far as you were concerned in homework 1, the shape was a **nuisance** parameter.


## A Classic Problem:


We'll start with an example of nuisance parameters that, in one form or another, dates all the way back to the posthumous [1763 paper](http://www.stat.ucla.edu/history/essay.pdf) written by Thomas Bayes himself. The particular version of this problem we'll study is borrowed from [Eddy 2004](ftp://selab.janelia.org/pub/publications/Eddy-ATG3/Eddy-ATG3-reprint.pdf).

The setting is a rather contrived game in which Alice and Bob bet on the outcome of a process they can't directly observe:

> Alice and Bob enter a room. Behind a curtain there is a billiard table, which they cannot see, but their friend Carol can. Carol rolls a ball down the table, and marks where it lands. Once this mark is in place, Carol begins rolling new balls down the table. If the ball lands to the left of the mark, Alice gets a point; if it lands to the right of the mark, Bob gets a point.  We can assume for the sake of example that Carol's rolls are unbiased: that is, the balls have an equal chance of ending up anywhere on the table.  **The first person to reach six points wins the game.**

Here ***the location of the mark (determined by the first roll) can be considered a nuisance parameter***.

It is unknown, and not of immediate interest, but it clearly must be accounted for when predicting the outcome of subsequent rolls. If the first roll settles far to the right, then subsequent rolls will favor Alice. If it settles far to the left, Bob will be favored instead.

Given this setup, here is the question we ask of ourselves:

> In a particular game, after eight rolls, Alice has five points and Bob has three points. What is the probability that Bob will go on to win the game?

Intuitively, you probably realize that because Alice received five of the eight points, the marker placement likely favors her. And given this, it's more likely that the next roll will go her way as well. 

And she has three opportunities to get a favorable roll before Bob can win; she seems to have clinched it.  But, **quantitatively**, what is the probability that Bob will squeak-out a win?


Someone following a classical frequentist approach might reason as follows:

To determine the result, we need an intermediate estimate of where the marker sits. We'll quantify this marker placement as a probability $p$ that any given roll lands in Alice's favor.  Because five balls out of eight fell on Alice's side of the marker, we can quickly show that the maximum likelihood estimate of $p$ is given by:

$$
\hat{p} = 5/8
$$

(This result follows in a straightforward manner from the [binomial likelihood](http://en.wikipedia.org/wiki/Binomial_distribution)). 

# 1.1 Under the assumptions of maximum likelihood, what is the probability of Bob winning the game - i.e. getting six points, and what are the odds 
# (i.e. $(1 - p_\text{Bob wins})/p_\text{Bob wins}$)

In [182]:
# following binomial likelihood, if the likelihood of Alice 
# getting a point is 5/8, then the probability of Bob getting a 
# point is 3/8. Bob has to get three points to win the game, an
# event which has total probability of: 
p_Alice = 5/8.
p_Bob = 1 - p_Alice
p_BWin = p_Bob**3
print("The probability of Bob winning from this point is %.2f%%.\n"%(p_BWin*100))

# The odds are: 
odds_BWin = (1 - p_BWin)/p_BWin

print("The odds of Bob winning are 1 in ~%.f.\n"%odds_BWin)

The probability of Bob winning from this point is 5.27%.

The odds of Bob winning are 1 in ~18.



We can also approach this problem from a Bayesian standpoint. This is slightly more involved, and requires us to first define some notation.

We'll consider the following random variables:

- $B$ = Bob Wins
- $D$ = observed data, i.e. $D = (n_A, n_B) = (5, 3)$
- $p$ = unknown probability that a ball lands on Alice's side during the current game

We want to compute $P(B~|~D)$; that is, the probability that Bob wins given our observation that Alice currently has five points to Bob's three.

The general Bayesian method of treating nuisance parameters is ***marginalization***, or integrating the joint probability over the entire range of the nuisance parameter. In this case, that means that we will first calculate the joint distribution
$$
P(B,p~|~D)
$$
and then marginalize over $p$ using the following identity:
$$
P(B~|~D) \equiv \int_{-\infty}^\infty P(B,p~|~D) {\mathrm d}p
$$
This identity follows from the definition of conditional probability, and the law of total probability: that is, it is a fundamental consequence of probability axioms and will always be true. Even a frequentist would recognize this; they would simply disagree with our interpretation of $P(p)$ as being a measure of uncertainty of our own knowledge.

To compute this result, we will manipulate the above expression for $P(B~|~D)$ until we can express it in terms of other quantities that we can compute.

We'll start by applying the following definition of [conditional probability](http://en.wikipedia.org/wiki/Conditional_probability#Definition) to expand the term $P(B,p~|~D)$:

$$
P(B~|~D) = \int P(B~|~p, D) P(p~|~D) dp
$$

Next we use [Bayes' rule](http://en.wikipedia.org/wiki/Bayes%27_theorem) to rewrite $P(p~|~D)$:

$$
P(B~|~D) = \int P(B~|~p, D) \frac{P(D~|~p)P(p)}{P(D)} dp
$$

Finally, using the same probability identity we started with, we can expand $P(D)$ in the denominator to find:

$$
P(B~|~D) = \frac{\int P(B~|~p,D) P(D~|~p) P(p) dp}{\int P(D~|~p)P(p) dp}
$$

Now the desired probability is expressed in terms of three quantities that we can compute. Let's look at each of these in turn:
<small>
- $P(B~|~p,D)$: This term is exactly the frequentist likelihood we used above. In words: given a marker placement $p$ and the fact that Alice has won 5 times and Bob 3 times, what is the probability that Bob will go on to six wins?  
    
    
- $P(D~|~p)$: this is another easy-to-compute term. In words: given a probability $p$, what is the likelihood of exactly 5 positive outcomes out of eight trials? The answer comes from the well-known [Binomial distribution](http://en.wikipedia.org/wiki/Binomial_distribution)
    
    
- $P(p)$: this is our prior on the probability $p$. By the problem definition, we can assume that $p$ is evenly drawn between 0 and 1.  That is, $P(p) \propto 1$, and the integrals range from 0 to 1.</small>

# 1.2 Evaluate that integral and get an expression in terms of $p$

First, the probability that Bob will go on to six wins given that we have a marker placement $p$ and the data $D$ is simply $(1-p)^3$. We have 3 more trials, and if Alice wins a single round then Bob loses. She must therefore lose every trial for Bob to win; therefore, 

$$ P(B|p,D) = (1 - p)^3$$

Next, the probability that Alice wins exactly 5 rounds out of 8 is: 

$$ P(D|p) = {8 \choose 5} p^5 (1-p)^3$$

(winning exactly five means losing exactly three, so we sum all possible permutations where this occurs). 

Finally, our prior is flat, so $P(p)$ is actually independent of $p$. That means we can pull it out of the numerator and denominator, and it will cancel out. Therefore, our integral becomes: 

$$
P(B|D) = \frac{\int_0^1 (1-p)^3 {8 \choose 5} p^5 (1-p)^3 dp}{\int_0^1 {8 \choose 5} p^5 (1-p)^3 dp} = \frac{\int_0^1 (1-p)^6  p^5 dp}{\int_0^1 (1-p)^3 p^5 dp}
$$



The integrals you find might look a bit difficult, but they are just special cases of the [Beta Function](http://en.wikipedia.org/wiki/Beta_function):
$$
\beta(n, m) = \int_0^1 (1 - p)^{n - 1} p^{m - 1}
$$

scipy has an implementation of this in `scipy.special`.

# 1.3 Evaluate your expression numerically and compute the probability and the odds that Bob wins

In [188]:
from scipy.special import beta 

# fraction is integral of beta(6, 7)/ integral of beta(6,4)
prob_BobWins = beta(6,7)/beta(6,4)
print("The probability that Bob wins (with a flat prior) is %.2f%%.\n"% (prob_BobWins*100))

# The odds are: 
odds_BWin = (1 - prob_BobWins)/prob_BobWins

print("The odds of Bob winning are 1 in ~%.1f.\n"%odds_BWin)

The probability that Bob wins (with a flat prior) is 9.09%.

The odds of Bob winning are 1 in ~10.0.



# 1.4 Finally, lets check the result using an approach called Monte Carlo, where we simulate a bunch of games, and simply count the fraction of relevant games that Bob goes on to win. The current problem is especially simple because so many of the random variables involved are uniformly distributed. 

* Simulate 100,000 random p between 0 and 1 - this will be a 1D array
* given each p, to win the game needs *at most* 11 rolls - simulate 11 rolls for each p - this will be a 2D array
* count the cumultative wins for Alice and Bob at each roll - this is a 2D array
* determine which games has Bob with three points by the end of game 8 - this is a 1D array
* considering only these games, find the number of games which Bob won (i.e. has six points at the end of game 11)
* compute the total probability that Bob won, and the odds that Bob won.


## You don't need anything more than `numpy` to do this - in particular `numpy.random` and `numpy.cumsum` to do this.
No for loops.

In [36]:
import numpy as np
# total number of games
N = 100000
# generate our random probabilities, our nuisance parameter!
p = np.random.uniform(0,1,N)
# the number of games we're going to play
n = 11
# generate our rolls
rolls = np.random.binomial(1,p,size=(n,N))
# determine how many points Bobs has scored at each point
runningWins = np.cumsum(rolls,axis=0)
# subset runningWins where the 8th entry is 3 (that's our data)
bobGames = runningWins[:,np.where(runningWins[7,:] == 3)]
# subset the games where Bob has 3 wins by game 8 where Bob wins by the end
bobWins = np.sum(bobGames[-1,:,:] == 6)

In [35]:
# we get that the probability that Bob wins given he has
# 3 wins by game 8 is: 

applicable_games = np.shape(bobGames)[2]
print("The probability that Bob wins, \
given he has 3 wins by game 8, is %.2f%%.\n" % (bobWins/applicable_games*100))

# The odds are: 

p_BWin = (bobWins/applicable_games)
odds_BWin = (1 - p_BWin)/p_BWin

print("The odds of Bob winning are 1 in ~%.1f.\n"%odds_BWin)

The probability that Bob wins, given he has 3 wins by game 8, is 9.11%.

The odds of Bob winning are 1 in ~10.0.



The Bayesians reign supreme! 

## 1.5 Why do the two results disagree?

The results disagree because the first approach was the frequentist approach, where we use the maximum likelihood estimate on the probability that the roll will land in Alice's favor. The Bayesian framework continued to treat this value as unknown, subject to the constraints of the data; in contrast, the frequentist approach fixed the value of p as the data value. We *know* that the probability $p$ is not necessarily 5/8 solely because Alice won on 5 trials and lost on 3. It seems like, for this reason, Bayesian statistics give better results if we choose a prior that makes sense (even a uniform prior, as we used). 